In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import sys

if not os.path.join(os.path.abspath(''), '../') in sys.path:
    sys.path.append(os.path.join(os.path.abspath(''), '../'))
    
from utils.classification import load_features, load_data
from utils.meta import *
import matplotlib
import warnings
import seaborn as sns
from matplotlib.patches import Circle
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.patches as mpatches

In [2]:
poses = load_data('../processed_dataset/raw_data.pkl')
features = load_features('../processed_dataset/binned_features.pkl')

In [4]:
import csv

selected_sex = 'Male'
selected_group = 'Group1'
selected_condition = 'Condition1'
selected_file = 'file0'

selected_data = features[selected_sex][selected_group][selected_condition][selected_file]

csv_file_path = ('../processed_dataset/selected_data.csv')

# Write selected_data to CSV file
with open(csv_file_path, 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)
    
    # Write each row to the CSV file
    for row in selected_data:
        csv_writer.writerow(row)

print("CSV file saved successfully.")

CSV file saved successfully.


NameError: name 'df' is not defined

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from utils.classification import load_features  # Importing the function to load features

# Step 1: Load Data from Pickle File
features = load_features('../processed_dataset/binned_features.pkl')

# Step 2: Select Specific Data
selected_sex = 'Male'
selected_group = 'Group1'
selected_condition = 'Condition1'
selected_file = 'file1'

selected_data = features[selected_sex][selected_group][selected_condition][selected_file]

# Step 3: Data Preprocessing (if necessary)
# Depending on the structure of selected_data, you may need to perform additional preprocessing.

# Step 4: Kinematic Analysis (if necessary)
# Perform your kinematic analysis here on the selected data.

# Step 5: Visualization
# Visualize the kinematic data using Matplotlib or other libraries

plt.figure(figsize=(10, 6))
for bp in range(0, 20): 
    plt.plot(selected_data[f'x{bp}'], selected_data[f'y{bp}'], label=f'Body Part {bp}')
plt.xlabel('X Coordinate')
plt.ylabel('Y Coordinate')
plt.title('Trajectories of Body Parts (Selected Data)')
plt.legend()
plt.grid(True)
plt.show()


In [6]:
import pandas as pd
import matplotlib.pyplot as plt
from utils.classification import load_features  # Importing the function to load features

# Step 1: Load Data from Pickle File
features = load_features('../processed_dataset/binned_features.pkl')

# Step 2: Select Specific Data
selected_sex = 'Male'
selected_group = 'Group1'
selected_condition = 'Condition1'
selected_file = 'file1'

# Assuming features is a dictionary
selected_data = features[selected_sex][selected_group][selected_condition][selected_file]

# Step 3: Data Preprocessing (if necessary)
# Depending on the structure of selected_data, you may need to perform additional preprocessing.

# Step 4: Access Specific Body Part Data
body_part_index = 0  # Assuming body parts are indexed from 0
x_displacement_column_index = 1  # Assuming x-displacement data is in column 1
y_displacement_column_index = 2  # Assuming y-displacement data is in column 2

body_part_data = selected_data.iloc[:, [x_displacement_column_index, y_displacement_column_index]]

# Step 5: Visualization
# Visualize the kinematic data using Matplotlib or other libraries

plt.figure(figsize=(8, 6))
plt.plot(body_part_data.iloc[:, 0], body_part_data.iloc[:, 1], label='Body Part 0')
plt.xlabel('X Displacement')
plt.ylabel('Y Displacement')
plt.title('Displacement of Body Part 0')
plt.legend()
plt.grid(True)
plt.show()


AttributeError: 'list' object has no attribute 'iloc'

df_file0

In [9]:
# Assuming 'feature_index' is the index of the feature you want to analyze
feature_index = 0

feature_values = [data[feature_index] for data in group_data]
average_feature_value = np.mean(feature_values)

UFuncTypeError: ufunc 'add' did not contain a loop with signature matching types (dtype('<U1'), dtype('<U1')) -> None

In [ ]:
# Assuming st.session_state['annotated_behaviors'] is a list of behaviors
st.session_state['annotated_behaviors'] = ['behavior1', 'behavior2', 'behavior3']

# Assuming bodypart is the index of the body part you're interested in
bodypart_index = 0

# Assuming framerate is 10
framerate = 10

# Call the get_avg_kinematics function
behavior, behavioral_start_time, behavior_duration, bout_disp_all, bout_duration_all, bout_avg_speed_all = get_avg_kinematics(
    features_data['predict'],
    pose_data['pose'],
    bodypart_index,
    framerate
)

# Use the results for further analysis or visualization

In [7]:
def get_avg_kinematics(predict, pose, bodypart, framerate=10):
    pose_estimate = pose[:, bodypart * 2:bodypart * 2 + 2]
    group_start = [0]
    group_start = np.hstack((group_start, np.where(np.diff(predict) != 0)[0] + 1))
    group_end = [len(predict) - 1]
    group_end = np.hstack((np.where(np.diff(predict) != 0)[0], group_end))

    bout_i_index = [np.arange(group_start[i], group_end[i]+1) for i in range(len(group_start))]
    # limiting to just the behavior bodypart x,y (10Hz) from start to end
    bout_pose_bodypart_i = [pose_estimate[bout_i_index[i], :] for i in range(len(bout_i_index))]

    behavior = predict[group_start]
    behavior_duration = np.hstack((np.diff(group_start), len(predict) - group_start[-1] + 1)) / framerate
    behavioral_start_time = group_start / framerate
    # kinematics portion
    bout_disp_all = {key: [] for key in st.session_state['annotated_behaviors']}
    bout_duration_all = {key: [] for key in st.session_state['annotated_behaviors']}
    bout_avg_speed_all = {key: [] for key in st.session_state['annotated_behaviors']}
    for b, behav in enumerate(st.session_state['annotated_behaviors']):
        behavior_j_bodypart_i_pose = []
        behavior_j_bout_duration = []
        behavior_index = np.where(behavior == b)[0]
        for instance in range(len(behavior_index)):
            # if behavior_duration[behavior_index][instance] > .1:
            behavior_j_bodypart_i_pose.append(bout_pose_bodypart_i[behavior_index[instance]])
            behavior_j_bout_duration.append(behavior_duration[behavior_index][instance])
        bout_avg_speed = []
        bout_duration = []
        bout_disp = []
        for n in range(len(behavior_j_bodypart_i_pose)):
            data_n_len = len(behavior_j_bodypart_i_pose[n])
            disp_list = []
            # going through the duration of each bout that's over 0.1s
            for r in range(data_n_len):
                if r < data_n_len - 1:
                    disp = []
                    # going through x and y
                    for c in range(0, behavior_j_bodypart_i_pose[n].shape[1], 2):
                        disp.append(
                            np.linalg.norm(behavior_j_bodypart_i_pose[n][r + 1, c:c + 2] -
                                           behavior_j_bodypart_i_pose[n][r, c:c + 2]))
                    disp_list.append(disp)
            disp_r = np.array(disp_list)
            disp_feat = np.array(disp_r)
            bout_disp.append(np.mean(disp_feat))
            bout_duration.append(behavior_j_bout_duration[n])
            bout_avg_speed.append(np.sum(disp_feat) / behavior_j_bout_duration[n])
        bout_disp_all[behav] = bout_disp
        bout_duration_all[behav] = bout_duration
        bout_avg_speed_all[behav] = bout_avg_speed

    return behavior, behavioral_start_time, behavior_duration, bout_disp_all, bout_duration_all, bout_avg_speed_all